In [1]:
import myosuite
import gym
import skvideo.io
import numpy as np
import os

RS:> Registering Myo Envs
myoSarFingerReachFixed-v0, myoFatFingerReachFixed-v0, myoSarFingerReachRandom-v0, myoFatFingerReachRandom-v0, myoSarElbowPose1D6MFixed-v0, myoFatElbowPose1D6MFixed-v0, myoSarElbowPose1D6MRandom-v0, myoFatElbowPose1D6MRandom-v0, myoSarElbowPose1D6MExoFixed-v0, myoFatElbowPose1D6MExoFixed-v0, myoSarElbowPose1D6MExoRandom-v0, myoFatElbowPose1D6MExoRandom-v0, myoSarFingerPoseFixed-v0, myoFatFingerPoseFixed-v0, myoSarFingerPoseRandom-v0, myoFatFingerPoseRandom-v0, myoSarHandPoseFixed-v0, myoFatHandPoseFixed-v0, myoTTHandPoseFixed-v0, myoSarHandPose0Fixed-v0, myoFatHandPose0Fixed-v0, myoTTHandPose0Fixed-v0, myoSarHandPose1Fixed-v0, myoFatHandPose1Fixed-v0, myoTTHandPose1Fixed-v0, myoSarHandPose2Fixed-v0, myoFatHandPose2Fixed-v0, myoTTHandPose2Fixed-v0, myoSarHandPose3Fixed-v0, myoFatHandPose3Fixed-v0, myoTTHandPose3Fixed-v0, myoSarHandPose4Fixed-v0, myoFatHandPose4Fixed-v0, myoTTHandPose4Fixed-v0, myoSarHandPose5Fixed-v0, myoFatHandPose5Fixed-v0, myoTTHandPose5Fixed

In [2]:
from IPython.display import HTML
from base64 import b64encode
 
def show_video(video_path, video_width = 400):
   
  video_file = open(video_path, "r+b").read()
 
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video autoplay width={video_width} controls><source src="{video_url}"></video>""")


In [44]:
pth = '../../../myosuite/agents/baslines_NPG/'

policy = pth+"myoElbowPose1D6MExoRandom-v0/2022-02-26_21-16-27/36_env=myoElbowPose1D6MExoRandom-v0,seed=1/iterations/best_policy.pickle"

import pickle
pi = pickle.load(open(policy, 'rb'))


In [45]:
env = gym.make('myoElbowPose1D6MExoRandom-v0')

env.reset()


array([0.7141, 0.006 , 1.1644, 0.    , 0.    , 0.    , 0.    , 0.    ,
       0.    ])

In [50]:
# define a discrete sequence of positions to test
AngleSequence = [60, 30, 30, 60, 80, 80, 60, 30, 80, 30, 80, 60]
env.reset()
frames = []
for ep in range(len(AngleSequence)):
    print("Ep {} of {} testing angle {}".format(ep, len(AngleSequence), AngleSequence[ep]))
    env.env.target_jnt_value = [np.deg2rad(AngleSequence[int(ep)])]
    env.env.target_type = 'fixed'
    env.env.weight_range=(0,0)
    env.env.update_target()
    for _ in range(40):
        frame = env.sim.render(width=400, height=400,mode='offscreen', camera_name=None)
        frames.append(frame[::-1,:,:])
        o = env.get_obs()
        a = pi.get_action(o)[0]
        next_o, r, done, ifo = env.step(a) # take an action based on the current observation
env.close()

os.makedirs('videos', exist_ok=True)
# make a local copy
skvideo.io.vwrite('videos/exo_arm.mp4', np.asarray(frames),outputdict={"-pix_fmt": "yuv420p"})


Ep 0 of 12 testing angle 60
Ep 1 of 12 testing angle 30
Ep 2 of 12 testing angle 30
Ep 3 of 12 testing angle 60
Ep 4 of 12 testing angle 80
Ep 5 of 12 testing angle 80
Ep 6 of 12 testing angle 60
Ep 7 of 12 testing angle 30
Ep 8 of 12 testing angle 80
Ep 9 of 12 testing angle 30
Ep 10 of 12 testing angle 80
Ep 11 of 12 testing angle 60


In [51]:
show_video('videos/exo_arm.mp4')